# TT-41: Hull+MACD Confluence Signal Detection

This notebook demonstrates the `HullMacdEngine` signal detection engine.

1. Connect to InfluxDB and load SPX 5m candle data
2. Get prior day's close for indicator seeding
3. Feed candles through the engine and collect signals
4. Display signal table
5. Overlay signals on Hull+MACD chart

In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
import logging

import pandas as pd
import polars as pl
from datetime import datetime, timedelta, timezone
from zoneinfo import ZoneInfo

import influxdb_client

from tastytrade.common.logging import setup_logging
from tastytrade.connections import InfluxCredentials
from tastytrade.config import RedisConfigManager
from tastytrade.providers.market import MarketDataProvider
from tastytrade.providers.subscriptions import RedisSubscription
from tastytrade.messaging.models.events import CandleEvent

from tastytrade.analytics.engines import HullMacdEngine, TradeSignal
from tastytrade.analytics.indicators.momentum import macd
from tastytrade.analytics.visualizations.plots import (
    plot_macd_with_hull,
    VerticalLine,
)

In [3]:
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)

logging.getLogger().handlers.clear()
setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix="dev_signals",
    console=True,
    file=True,
)

2026-02-11 21:04:48 - INFO:root:62:Logging initialized - writing to ../logs/dev_signals_20260211.log


## Connect to Data Sources

In [4]:
config = RedisConfigManager(env_file="/workspace/.env")
config.initialize()

influxdb = influxdb_client.InfluxDBClient(
    url=InfluxCredentials(config=config).url,
    token=InfluxCredentials(config=config).token,
    org=InfluxCredentials(config=config).org,
)

subscription = RedisSubscription(config=RedisConfigManager())
await subscription.connect()

streamer = MarketDataProvider(subscription, influxdb)

2026-02-11 21:04:48 - INFO:tastytrade.config.manager:174:Initialized 52 variables from .env file in Redis
2026-02-11 21:04:48 - INFO:tastytrade.providers.subscriptions:72:Listening to Redis at redis://redis:6379/0


## Date Setup & Prior Day Close

In [5]:
date = datetime.now(ZoneInfo("America/New_York"))

if date < datetime(date.year, date.month, date.day, 9, 30, tzinfo=date.tzinfo):
    date -= timedelta(days=1)

while date.weekday() > 4:
    date -= timedelta(days=1)

market_open = datetime(date.year, date.month, date.day, 9, 30, tzinfo=date.tzinfo)
market_close = datetime(date.year, date.month, date.day, 16, 0, tzinfo=date.tzinfo)

start = market_open.astimezone(timezone.utc) - timedelta(minutes=30)
stop = market_close.astimezone(timezone.utc)

candle_symbol = "SPX{=5m}"

prior_date = market_open.date() - timedelta(days=1)
while prior_date.weekday() > 4:
    prior_date -= timedelta(days=1)

prior_day: CandleEvent = streamer.get_daily_candle(candle_symbol, prior_date)
print(f"Prior day close: {prior_day.close}")

Prior day close: 6941.81


## Download 5m Candles

In [6]:
candles_5m: pl.DataFrame = streamer.download(
    symbol=candle_symbol,
    start=start,
    stop=stop,
    debug_mode=True,
)

candles_5m = candles_5m.filter(
    (pl.col("open") != 0)
    & (pl.col("high") != 0)
    & (pl.col("low") != 0)
    & (pl.col("close") != 0)
)
print(f"Loaded {candles_5m.height} candles")

Loaded 78 candles


## Feed Candles Through HullMacdEngine

In [7]:
engine = HullMacdEngine()
engine.set_prior_close(candle_symbol, float(prior_day.close))

for row in candles_5m.iter_rows(named=True):
    event = CandleEvent(**row)
    engine.on_candle_event(event)

print(f"Signals detected: {len(engine.signals)}")

2026-02-11 21:04:48 - INFO:tastytrade.analytics.engines.hull_macd:306:TradeSignal: OPEN BEARISH SPX{=5m} at 2026-02-11T15:05:00 (trigger=confluence)
2026-02-11 21:04:48 - INFO:tastytrade.analytics.engines.hull_macd:306:TradeSignal: CLOSE BEARISH SPX{=5m} at 2026-02-11T15:50:00 (trigger=hull)
2026-02-11 21:04:48 - INFO:tastytrade.analytics.engines.hull_macd:306:TradeSignal: OPEN BULLISH SPX{=5m} at 2026-02-11T16:10:00 (trigger=confluence)
2026-02-11 21:04:48 - INFO:tastytrade.analytics.engines.hull_macd:306:TradeSignal: CLOSE BULLISH SPX{=5m} at 2026-02-11T16:55:00 (trigger=hull)
2026-02-11 21:04:48 - INFO:tastytrade.analytics.engines.hull_macd:306:TradeSignal: OPEN BEARISH SPX{=5m} at 2026-02-11T17:00:00 (trigger=confluence)
2026-02-11 21:04:48 - INFO:tastytrade.analytics.engines.hull_macd:306:TradeSignal: CLOSE BEARISH SPX{=5m} at 2026-02-11T17:10:00 (trigger=macd)
2026-02-11 21:04:48 - INFO:tastytrade.analytics.engines.hull_macd:306:TradeSignal: OPEN BULLISH SPX{=5m} at 2026-02-11T17

Signals detected: 14


## Signal Table

In [8]:
if engine.signals:
    signal_data = [
        {
            "time": s.start_time,
            "type": s.signal_type,
            "direction": s.direction,
            "trigger": s.trigger,
            "close_price": s.close_price,
            "hull_dir": s.hull_direction,
            "hull_value": round(s.hull_value, 2),
            "macd_value": round(s.macd_value, 4),
            "macd_signal": round(s.macd_signal, 4),
            "macd_hist": round(s.macd_histogram, 4),
        }
        for s in engine.signals
    ]
    df_signals = pd.DataFrame(signal_data)
    display(df_signals)
else:
    print("No signals detected for this session.")

,time,type,direction,trigger,close_price,hull_dir,hull_value,macd_value,macd_signal,macd_hist
0,2026-02-11 15:05:00,OPEN,BEARISH,confluence,6948.23,Down,6973.12,5.3139,5.4582,-0.1443
1,2026-02-11 15:50:00,CLOSE,BEARISH,hull,6945.58,Up,6919.37,-4.4478,-2.8894,-1.5584
2,2026-02-11 16:10:00,OPEN,BULLISH,confluence,6949.13,Up,6934.39,-2.1100,-2.9132,0.8031
3,2026-02-11 16:55:00,CLOSE,BULLISH,hull,6941.15,Down,6950.23,0.6662,0.0539,0.6123
4,2026-02-11 17:00:00,OPEN,BEARISH,confluence,6936.31,Down,6948.10,-0.0626,0.0306,-0.0932
5,2026-02-11 17:10:00,CLOSE,BEARISH,macd,6944.30,Down,6944.72,0.0458,0.0271,0.0187
6,2026-02-11 17:20:00,OPEN,BULLISH,confluence,6951.49,Up,6944.63,0.8160,0.2198,0.5962
7,2026-02-11 18:00:00,CLOSE,BULLISH,hull,6947.14,Down,6956.47,2.2947,2.2726,0.0220
8,2026-02-11 18:05:00,OPEN,BEARISH,confluence,6941.14,Down,6954.41,1.3535,2.0888,-0.7353
9,2026-02-11 18:45:00,CLOSE,BEARISH,hull,6948.92,Up,6942.77,-0.0725,0.1789,-0.2515


## Chart with Signal Overlays

In [9]:
df_macd_5m = macd(
    candles_5m,
    prior_close=prior_day.close,
    fast_length=12,
    slow_length=26,
    macd_length=9,
)

signal_lines = [
    VerticalLine(
        start_time=s.start_time,
        color="#55A868" if s.direction == "BULLISH" else "#8C8C8C",
        line_dash="dot",
        line_width=0.5,
        label=f"{s.signal_type} {s.direction}",
        opacity=0.4,
    )
    for s in engine.signals
]

plot_macd_with_hull(
    df_macd_5m,
    pad_value=prior_day.close,
    start_time=start,
    end_time=stop + timedelta(minutes=15),
    vertical_lines=signal_lines,
)

## Historic Signals from InfluxDB

Query persisted `TradeSignal` records from InfluxDB and overlay them on the chart
using `download_signals()` → `to_vertical_line()` conversion.

In [ ]:
historic_signals = streamer.download_signals(
    symbol=candle_symbol, start=start, stop=stop
)
print(f"Retrieved {len(historic_signals)} signals from InfluxDB")

historic_lines = [s.to_vertical_line() for s in historic_signals]

plot_macd_with_hull(
    df_macd_5m,
    pad_value=prior_day.close,
    start_time=start,
    end_time=stop + timedelta(minutes=15),
    vertical_lines=historic_lines,
)